## Examples

In [2]:
from gensim.models import FastText
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
ft = FastText(sentences, min_count=1, size=10)
from fse.models import Average
from fse import IndexedList
model = Average(ft)
model.train(IndexedList(sentences))
model.sv.similarity(0,1)

0.48041457

## word embedding and SIF model

In [3]:
from gensim.models import KeyedVectors
import pandas as pd
from fse.models import SIF
import gensim
from gensim.models import Word2Vec
import os
wemb_model = KeyedVectors.load_word2vec_format("data/crawl-300d-2M.vec")
# PATH_TO_WORD2VEC = os.path.expanduser("data/GoogleNews-vectors-negative300.bin")
# word2vec = gensim.models.KeyedVectors.load_word2vec_format(PATH_TO_WORD2VEC, binary=True)
model = SIF(wemb_model, components=2, lang_freq="en")

C:\Users\70473\Anaconda3\lib\site-packages\fse\models\base_s2v.py:114: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  warnings.warn(


In [4]:
raw = pd.read_pickle("data.pkl")

## train sentence embedding

In [5]:
from fse import SplitIndexedList
indexed_descriptions = SplitIndexedList(raw["description"].values)
model.train(indexed_descriptions)

(661704, 43249024)

## Database

In [6]:
import sys
import pandas as pd
import json
from pymongo import MongoClient
class my_mongodb:
    import pandas as pd
    def __init__(self, hostname='localhost', db_port=27017):

        self.hostname = hostname
        self.db_port = db_port
        self.my_mongo_client = MongoClient(self.hostname, self.db_port)
        
    def df2mongo(self, df_data, db_name, form_name):


        def df2bson(df):

            data = json.loads(df.T.to_json()).values()
            return data

        my_db = self.my_mongo_client[db_name]
        bson_data = df2bson(df_data)
        my_posts = my_db[form_name]
        result = my_posts.insert_many(bson_data)
        return result

    def collection2df(self, db_name, collection_name, query={}, no_id=True):

        """查询数据库，导出DataFrame类型数据
        （db_name：数据库名 collection_name：集合名 
         query：查询条件式 no_id：不显示ID,默认为不显示ID）"""

        db = self.my_mongo_client[db_name]
        collection = self.my_mongo_client[db_name][collection_name]
        cursor = collection.find(query)
        df = pd.DataFrame(list(cursor))
        if no_id:
            del df['_id']
        return df

## Similarity computing

In [70]:
ls = []
for i in name_ls_travel.index:
    ls.append((i,model.sv.similarity(6887,i).round(3)))

In [74]:
sorted(ls, key=lambda x: x[1], reverse = True)

[(6887, 1.0),
 (129495, 0.561),
 (212182, 0.528),
 (634546, 0.525),
 (600840, 0.513),
 (145346, 0.502),
 (479554, 0.5),
 (112401, 0.495),
 (187092, 0.494),
 (327563, 0.49),
 (141840, 0.485),
 (186793, 0.48),
 (398041, 0.476),
 (32681, 0.473),
 (62132, 0.47),
 (304816, 0.47),
 (324878, 0.468),
 (562522, 0.464),
 (8216, 0.463),
 (55713, 0.463),
 (59422, 0.462),
 (108533, 0.461),
 (24686, 0.46),
 (153973, 0.46),
 (421638, 0.46),
 (639925, 0.459),
 (169282, 0.451),
 (109103, 0.45),
 (2228, 0.448),
 (55563, 0.448),
 (77258, 0.448),
 (225014, 0.445),
 (405392, 0.444),
 (92317, 0.443),
 (169947, 0.441),
 (393135, 0.44),
 (72014, 0.439),
 (228306, 0.435),
 (407989, 0.433),
 (26127, 0.432),
 (313826, 0.43),
 (16591, 0.429),
 (172723, 0.429),
 (296895, 0.429),
 (140437, 0.428),
 (89129, 0.427),
 (295416, 0.427),
 (304797, 0.422),
 (140994, 0.42),
 (278269, 0.42),
 (383222, 0.42),
 (50776, 0.419),
 (66781, 0.419),
 (515410, 0.419),
 (240385, 0.418),
 (210693, 0.417),
 (594780, 0.417),
 (23337, 0.

In [65]:
name_ls_travel = raw[raw.apply(lambda x: "Travel" in x["category_list"],axis = 1)]

In [ ]:
def first_return_searh_result(query_company,database,data,wem_model):
    set_a = data[data["name"] == query_company]["category_coding"].values[0]
    search_data = data[data["category_coding"].apply(lambda x:Search_coms(x,set_a))]
    collection = database[query_company]
    print("Please wait for generating all the competitors")
    for search_company in search_data["name"].values[:10]:
        pair = {}
        try:
            sim, com_des_one,com_des_two = return_similarity(data,query_company,search_company,wem_model,False,"cos")
#             print(query_company,search_company)
            cat_sim = return_cat_sim(data,query_company,search_company)
            size_diff = data[data["name"] == query_company]["status_coding"].values[0] - data[data["name"] == search_company]["status_coding"].values[0]
        except:
            pass
        pair["name_query"] = query_company
        pair["name_result"] = search_company
        pair["query_description"] = com_des_one
        pair["result_description"] = com_des_two
        pair["text_similarity"] = float(sim)
        pair["cat_similarity"] = float(cat_sim)
        pair["size_difference"] = int(size_diff)
        collection.insert_one(pair)
    print("************Data is prepared**************")

In [ ]:
def startup_competitor_search_engine(db_name,data,word2vec):
    query_company = input("please input query company name in the Crunchdatabase.e.g. Airbnb, Coinbase"+"\n" +"You are searching for competitors of ")
    query_company = query_company.lower().capitalize()
    mango = my_mongodb()
    client = mango.my_mongo_client
    database = client[db_name]
    collist = database.list_collection_names()
    collection = database[query_company]
    if query_company in collist: 
        print("This is not the first query for this company, results come out soon......"+"\n")
        result_df = mango.collection2df( "dissertation", query_company, query={}, no_id=True)
        final_result = pd.DataFrame(sorted(result_df.values, key=lambda x: 0.8*x[4]+0.2*x[5], reverse = True),columns=['name_query', 'name_result', 'query_description', 'result_description',
       'text_similarity', 'cat_similarity', 'size_difference'])
        return final_result
    else:
        print("This is the first query for this company......"+"\n")
        first_return_searh_result(query_company,database,data,word2vec)
        result_df = mango.collection2df( "dissertation", query_company, query={}, no_id=True)
        final_result = pd.DataFrame(sorted(result_df.values, key=lambda x: 0.8*x[4]+0.2*x[5], reverse = True),columns=['name_query', 'name_result', 'query_description', 'result_description',
       'text_similarity', 'cat_similarity', 'size_difference'])
        return final_result

In [ ]:
startup_competitor_search_engine("Baseline",data,word2vec)